# Cutting Trajectories

In [1]:
import os
os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())

working dir /home/tales/dev/master/mdc_analysis


In [2]:
import pandas as pd
import numpy as np
import math
import copy

from bokeh.models import PrintfTickFormatter
from bokeh.io import output_notebook, show

from src.dao import csv_dao, objects_dao
from src.plot.basic_plot import plot_result
from src.similarity.extreme_travelers import sequence_report
from src.data_processment.input_data_version2 import InputDataManager
from src.utils.time_utils import human_time

from src.data_processment.trajectory_cutting import gaps_params, cut_traj_in_trips

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
output_notebook()

Loading BokehJS ...

## Loading User Data

In [5]:
# users_srg = objects_dao.load_all_stop_region_group_object()
# users_tags = objects_dao.load_users_tags_sequence(sr_stay_time_above_h=0.5)["orignal"]

In [6]:
users_gps_dir = os.listdir("outputs/user_gps/")
users_gps = {}

for user_gps_dir in users_gps_dir:
    user = user_gps_dir.split("_")[0]
    users_gps[user] = csv_dao.load_user_gps_csv(user)

## Gaps

How many gaps above 06h ? <br> 
How many gaps above 12h ? <br> 
How many gaps above 24h ? <br>
How many gaps above 48h ? <br>
How much time the user has participated?

In [7]:
def gap_missing_values(report):
    t2 = report.iloc[1:len(report)]["local_time"].astype(float).reset_index(drop=True)
    t1 = report.iloc[0:len(report) - 1]["local_time"].astype(float).reset_index(drop=True)
    gaps = t2 - t1
    gaps.index = report.iloc[1:len(report)]["local_time"]
    head = pd.Series([None], index=[t1.iloc[0].item()])
    return head.append(gaps)

def gap_amount(gap, min_time_h):
    present_data_sum = gap[gap <= 60 * 60 * min_time_h].sum()
    missing_data_sum = gap[gap > 60 * 60 * min_time_h].sum()
    
    return {"missing_data_sum": missing_data_sum, "present_data_sum": present_data_sum}

def user_time_participation(report):
    first = report["local_time"].min()
    last = report["local_time"].max()
    
    n_days = (last - first) / (60*60*24)
    
    return {"n_days": n_days,
            "n_years": n_days / 365.25,
            "datetime_start": human_time(first, datetime_format='%d-%m-%Y %H:%M:%S')["datetime"],
            "datetime_end": human_time(last, datetime_format='%d-%m-%Y %H:%M:%S')["datetime"]}
    
def gaps_summary(gaps, min_time_h):
    selection = gaps[gaps > 60 * 60 * min_time_h]
    return {"n_gaps": len(selection),
            "median_sec": selection.median(),
            "min_time_gap": min_time_h}

def merge_dicts(dicts):
    final_dict = {}
    
    for d in dicts:
        
        for k in d.keys():
            final_dict[k] = d[k]

    return final_dict

#gaps per week
def gaps_week(gaps, user_data, min_time_h): 
    gaps_df = pd.DataFrame({"gaps": gaps.tolist(), "local_time": user_data["local_time"][0:-1].tolist()})
    gaps_df["week"] = ((gaps_df["local_time"] - gaps_df["local_time"].min()) / (60*60*24*7)).apply(math.floor)
    
    return gaps_df[gaps_df["gaps"] > 60*60*min_time_h]

# Cutting Trajectories

In [8]:
users_gps["5937"].head()

,latitude,longitude,speed,horizontal_accuracy,vertical_accuracy,speed_accuracy,altitude,horizontal_dop,vertical_dop,db_key,userid,tz,time,type,local_time
4052,46.31,6.97,nan,82.59,55.50,nan,450.50,7.23,5.63,81673215,5937,-3600,1253637575,gps,1253633975
12127,46.31,6.97,54.00,81.21,62.00,10.08,467.50,4.77,3.15,81673216,5937,-3600,1253637585,gps,1253633985
18542,46.31,6.97,87.05,42.90,34.50,3.42,467.00,4.77,3.14,81673217,5937,-3600,1253637595,gps,1253633995
7521,46.31,6.96,68.29,99.69,66.00,1.98,465.50,7.15,5.53,81673218,5937,-3600,1253637605,gps,1253634005
643,46.32,6.96,48.85,33.83,30.50,5.69,464.50,2.41,2.29,81673223,5937,-3600,1253637615,gps,1253634015


In [9]:
# def gaps_params(user_gps_data, gap_tresh_minutes):
#     start_stop_gaps = []
    
#     gaps_data = gap_missing_values(user_gps_data).to_frame().reset_index().rename({0: "gap_time", "index": "stop"}, axis=1)
    
#     gaps_data["start"] = [None] + gaps_data.iloc[0 : len(gaps_data) - 1]["stop"].tolist()
#     gaps_data["start"] = gaps_data["start"].replace({np.NaN: None})
    
#     selected_gaps = copy.deepcopy(gaps_data[gaps_data["gap_time"] > gap_tresh_minutes * 60])
    
#     selected_gaps["user_id"] = user_gps_data["userid"].drop_duplicates().item()
    
#     return selected_gaps[["user_id", "gap_time", "start", "stop"]]
    
gaps_times = gaps_params(users_gps["5937"], 30)
gaps_times.head(6)

,user_id,gap_time,start,stop
245,5937,15651.00,1253639643.00,1253655294.00
253,5937,26816.00,1253655703.00,1253682519.00
329,5937,865123.00,1253683561.00,1254548684.00
844,5937,4801.00,1254557227.00,1254562028.00
1897,5937,56784.00,1254580460.00,1254637244.00
1976,5937,8506.00,1254638336.00,1254646842.00


In [10]:
# gaps_times.head(6).to_dict(orient="records")

# Applying Trajectories Cut To InputDataManager

In [11]:
data_manager = InputDataManager(use_cache=False)

Loading Users Sequence Report
Loading user_id: 6189 - 1 out of 163
Loading user_id: 5936 - 2 out of 163
Loading user_id: 6087 - 3 out of 163
Loading user_id: 5973 - 4 out of 163
Loading user_id: 6085 - 5 out of 163


In [12]:
# input_data = {}

# for version in data_manager.avaliable_versions():
#     print("version:", version)
#     input_data[version] = {}
#     for sr_min_time in [5,15,30]:
#         print("sr_min_time:", sr_min_time)
#         input_data[version][sr_min_time] = data_manager.get_input_data(version=version, sr_stay_time_minutes=sr_min_time)


In [13]:
user_id = "5936"

user_srg = objects_dao.load_stop_region_group_object(user_id)
user_gps_data = csv_dao.load_user_gps_csv(user_id)
gaps = gaps_params(user_gps_data, 60 * 12)

In [14]:
# gaps = gaps_params(user_gps_data, 60 * 48)
# gaps.iloc[100:102]

In [15]:
seq = user_srg.sequence_report(enrich_columns=True)
seq = seq[["sr_start_time", "sr_end_time", "tags", "start_date", "start_time", "end_date", "end_time"]]
seq["gap_mins"] = (seq["sr_end_time"] - seq["sr_start_time"]) / 60.
seq.head()

,sr_start_time,sr_end_time,tags,start_date,start_time,end_date,end_time,gap_mins
0,1253851684.00,1253852031.00,"[liquor_store, convenience_store, restaurant, ...",2009-09-25,04:08:04,2009-09-25,04:13:51,5.78
1,1253852071.00,1253867442.00,[WORK],2009-09-25,04:14:31,2009-09-25,08:30:42,256.18
2,1254042356.00,1254042912.00,"[locality, political]",2009-09-27,09:05:56,2009-09-27,09:15:12,9.27
3,1254043814.00,1254044738.00,[lodging],2009-09-27,09:30:14,2009-09-27,09:45:38,15.40
4,1254150133.00,1254194904.00,[HOME],2009-09-28,15:02:13,2009-09-29,03:28:24,746.18


In [16]:
# seq[(seq["sr_start_time"] > 1256994464.00) & (seq["sr_start_time"] < 1257142762.00)].head()

In [17]:
cut = cut_traj_in_trips(srg_sequence_report=user_srg.sequence_report(enrich_columns=True), gaps=gaps)
pd.Series(cut).apply(len).value_counts()

KeyError: 'end'